In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna, joblib, matplotlib, os, pyfolio, sys
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from pyfolio import timeseries

matplotlib.use('Agg')
%matplotlib inline
import datetime
from stockstats import StockDataFrame as Sdf
from QuantumAI.config import config
from QuantumAI.config import config_tickers
from QuantumAI.preprocessors.yahoodownloader import YahooDownloader
from QuantumAI.preprocessors.preprocessors import FeatureEngineer, data_split
# from QuantumAI.env.env_portfolio import StockPortfolioEnv
# from QuantumAI.env.env_portfolioSPX import StockPortfolioEnv
from QuantumAI.env.env_portfolio_original import StockPortfolioEnv
from QuantumAI.env.env_portfolioSPXvsCash import SPXvsCASH
from QuantumAI.models.models import DRLAgent
from QuantumAI.plot.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts, show_weight_distribution
from QuantumAI.preprocessors.data_processor import DataProcessor
from QuantumAI.preprocessors.processor_yahoofinance import YahooFinanceProcessor
from QuantumAI.hyper_opt.hyperopt1 import optimize

/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/pyfolio/pos.py:25: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
bbg_full_start_date="1991-01-02"
full_end_date="2023-07-25"
bbg_train_start_date="1991-01-02"
bbg_train_end_date="2015-12-31"
bbg_validate_start_date="2016-01-01"
bbg_validate_end_date="2020-12-31"
bbg_test_start_date="2021-01-01"
bbg_test_end_date="2023-07-25"
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [3]:
df =  pd.read_pickle("pickle.pkl")

In [4]:
train = data_split(df, bbg_train_start_date, bbg_train_end_date)
validate = data_split(df, bbg_validate_start_date, bbg_validate_end_date)
test = data_split(df, bbg_test_start_date, bbg_test_end_date)

In [5]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 2, State Space: 2


In [6]:
all_techs = ['RSI14', 'RSI30', 'RSI3',
       'MA200', 'MA50', 'MA20', 'PUT Index', 'RXM Index',
       'USGG2YR Index', 'USGG10YR Index', 'MOVE Index', 'VIX Index',
       'NWHLSENY Index', 'NWHLSEND Index', 'USGG5YR Index', 'SFFRNEWS Index',
       'US10-US2Y', 'macd', 'boll_ub', 'boll_lb',  'cci_30', 'dx_30',
       'close_30_sma', 'close_60_sma']
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension,
    # "eco_indicator_list": eco_indicator_list, 
    # "tech_indicator_list": new_tech_indics, 
    "tech_indicator_list": all_techs, 
    # "tech_indicator_list": config.INDICATORS, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    # "cov": False
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)
# !FLO New validation set
e_validate_gym = StockPortfolioEnv(df = validate, **env_kwargs)

Observation space Box(-inf, inf, (26, 2), float32)
state space 2
Observation space Box(-inf, inf, (26, 2), float32)
state space 2


In [7]:
env_train, _ = e_train_gym.get_sb_env()
env_validate, _ = e_validate_gym.get_sb_env()

/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning:

You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.



In [8]:
agent = DRLAgent(env = env_train)
A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS, tensorboard_log='/Users/floriankockler/Documents/GitHub.nosync/TrainedModels/TensorboardLogs/')

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cpu device


In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c_etf',
                                total_timesteps=50000)